# **Part II: Analysis**
team member: Shaoying Zheng, Zhongrui Ning, Xiao Pu


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

In [3]:
# load data
df = pd.read_csv('./Projectdata/job_company_pair.csv')
df.head()

,job_id,company_id,skill_abr,skill_name,industry_id,max_salary,med_salary,min_salary,pay_period,industry_name,name,company_size,country,employee_count,follower_count,employee_follower_ratio,annual_min_salary,annual_max_salary,annual_med_salary,skill_category
0,921716,2774458.0,MRKT,Marketing,44,20.0,NaN,17.0,HOURLY,Real Estate,Corcoran Sawyer Smith,2.0,US,402,2351,0.170991,35360.0,41600.0,61568.0,Creative
1,921716,2774458.0,SALE,Sales,44,20.0,NaN,17.0,HOURLY,Real Estate,Corcoran Sawyer Smith,2.0,US,402,2351,0.170991,35360.0,41600.0,61568.0,Business
2,10998357,64896719.0,MGMT,Management,32,65000.0,NaN,45000.0,YEARLY,Restaurants,The National Exemplar,1.0,US,15,40,0.375000,45000.0,65000.0,88000.0,Administration
3,10998357,64896719.0,MNFC,Manufacturing,32,65000.0,NaN,45000.0,YEARLY,Restaurants,The National Exemplar,1.0,US,15,40,0.375000,45000.0,65000.0,88000.0,Other
4,23221523,766262.0,OTHR,Other,9,175000.0,NaN,140000.0,YEARLY,Law Practice,"Abrams Fensterman, LLP",2.0,US,222,2427,0.091471,140000.0,175000.0,252000.0,Other


## Descriptive Statistics: 

Firstly, let's check the columns of our data

In [4]:
df.columns

Index(['job_id', 'company_id', 'skill_abr', 'skill_name', 'industry_id',
       'max_salary', 'med_salary', 'min_salary', 'pay_period', 'industry_name',
       'name', 'company_size', 'country', 'employee_count', 'follower_count',
       'employee_follower_ratio', 'annual_min_salary', 'annual_max_salary',
       'annual_med_salary', 'skill_category'],
      dtype='object')

In those columns, `job_id` is the unique value for each job
For the purpose of our analysis, which aiming at job hunting recommendation, we would focus on those following columns: 
- Salary of the job: described by `annual_min_salary`, `annual_med_salary` and `annual_max_salary`
- Skills the job needs: described by `skill_abr`, `skill_name` specifically and `skill_catogory` generally. 
- Size and popularity of the company：described by `employee_count`,`follower_count`, `company_size` and `employee_follower_ratio`
- Industry fields: described by 'industry_name' and 'industry_id'

What are the most common skills listed in job postings across various industries?
What we hope to learn: By identifying the most frequently mentioned skills, we hope to find some "universal" skills in this era.

How do job requirements vary across different industries?
What we hope to learn: We hope to identify the unique skills and qualifications required in different industries, which can help job seekers better understand the job market and make informed decisions.

How could job hunters with different background find suitable jobs?
What we hope to learn: We hope to build a model that can provide job recommendations based on the applicant's background information, such as education, skills, and industry preference. Also, we hope to explore how the model can adapt to incomplete information.
(暂时看一下问题的，之后要删掉)

### Salary description

`annual_min_salary`, `annual_med_salary` and `annual_max_salary` are all numerical float variables, each columns contains the salary description of the job, but not every job have those three kind of salary descriptions (minimum, median and maximum). 

In [5]:
df[['annual_min_salary', 'annual_med_salary', 'annual_max_salary']].describe().T

,count,mean,std,min,25%,50%,75%,max
annual_min_salary,307895.0,128614.925247,3.112777e+06,1.0,37960.0,69800.0,105930.0,499200000.0
annual_med_salary,307895.0,241940.591169,5.256339e+06,1.0,75712.0,130090.4,207200.0,856960000.0
annual_max_salary,307895.0,180584.920583,3.704833e+06,1.0,60250.7,98400.0,151840.0,572000000.0


- We handled the missing values in part 1, and filled all the missing values
- In Part I, we have briefly described the relationship between `annual_med_salary` and occupation type/company size. 
- We didn't use `annual_min_salary` and `annual_max_salary` before. 
- We may use `annual_med_salary` as the prediction variable. 

### Skills description

`skill_abr`, `skill_name` and `skill_category` are all text variables, contains the arbitrary of the skills, the full name of the skills and the category of the skills. 

In [10]:
df. skill_category.value_counts().describe()

count         9.000000
mean      34210.555556
std       40375.064517
min        1066.000000
25%        4552.000000
50%       28084.000000
75%       37179.000000
max      132184.000000
Name: count, dtype: float64

In [11]:
df. skill_category.value_counts().head(10)

skill_category
Other                      132184
Administration              48252
Business                    37179
Technical & Engineering     36144
Finance                     28084
Creative                    15942
Legal & Compliance           4552
Human Resources              4492
Logistics & Operations       1066
Name: count, dtype: int64

## Inferential Statistics

## Graphical Analysis

## Compartive Analysis

## Multivariate Analysis

## Synthesis

## Reflection